In [1]:
import numpy as np
import sys 
import os 
import do_mpc 
import matplotlib.pyplot as plt
import pyomo.environ as pyomo
from pyomo.opt import SolverFactory

<font color = 'orange'> Todo: Implement the "optimizing_battery_schedule_using_pyomo" solution in this Jupyter notebook, </br> see if I can mess around with the parameters to get it to work for my setting or not. Then see if I can include load shedding in some way - optimizing to minimize unserved kWh</font>

Objective = α * LoadSheddingObjective + β * ProfitObjective

<font color = 'orange'> Allow grid energy to be used to charge the battery?? Why not?? They do this in their optimization and it's a good idea </font> <br>
<font color = 'red'> Hmm but maybe not such a good idea because that places extra strain on the grid </font> <br>
But I can try it and then we can adjust later... </br>

Then optimize the capacity alongside scheduling in two stage optimization

In [91]:
class Battery: 
    def __init__(self, 
                 current_charge = 0.0,
                 capacity = 10.0,
                 charging_power_limit = 5.0,
                 discharging_power_limit = -5.0,
                 charging_efficiency = 0.90, 
                 discharging_efficiency = 0.95):
        self.current_charge = current_charge
        self.capacity = capacity
        self.charging_power_limit = charging_power_limit
        self.discharging_power_limit = discharging_power_limit
        self.charging_efficiency = charging_efficiency
        self.discharging_efficiency = discharging_efficiency
        
        self.max_SOC = 0.90 # minimum depth of discharge 
        self.min_SOC = 0.10 # maximum depth of discharge
batt = Battery()

# Generate Data

In [93]:
T = 24 

np.random.seed(42)
load = np.random.rand(T) * 50.0
pv =  np.random.rand(T) * 20.0
net_load = load - pv 
prices = np.random.rand(T)* 20.0 
feed_in_tariff = prices/2

posLoad = np.copy(net_load)
negLoad = np.copy(net_load)

for j, e in enumerate(net_load):
    if e > 0:
        negLoad[j] = 0
    else:
        posLoad[j] = 0
        
posLoadDict = dict(enumerate(posLoad))
negLoadDict = dict(enumerate(negLoad))

        

# Model 

In [94]:
model = pyomo.ConcreteModel()
model.Time = pyomo.RangeSet(0, T-1)
# Time indexed variables
model.SOC = pyomo.Var(model.Time, bounds=(0,batt.capacity), initialize=0) #0
model.posDeltaSOC = pyomo.Var(model.Time, initialize = 0) # Positive change in SOC
model.negDeltaSOC = pyomo.Var(model.Time, initialize = 0) # Negative change in SOC
model.posEInGrid = pyomo.Var(model.Time, bounds = (0, batt.charging_power_limit), initialize = 0) # energy from grid to battery
model.posEInPV = pyomo.Var(model.Time, bounds = (0, batt.charging_power_limit), initialize = 0) # energy from PV to battery
model.negEOutLocal = pyomo.Var(model.Time, bounds = (batt.discharging_power_limit, 0), initialize = 0) # energy from battery to local load
model.negEOutExport = pyomo.Var(model.Time, bounds = (batt.discharging_power_limit, 0), initialize = 0) # energy from battery to grid
model.posNetLoad = pyomo.Var(model.Time, initialize = posLoadDict) # positive net load
model.negNetLoad = pyomo.Var(model.Time, initialize = negLoadDict) # negative net load

# Boolean variables
model.Bool_char = pyomo.Var(model.Time, domain = pyomo.Boolean) # Charging
model.Bool_dis = pyomo.Var(model.Time, domain = pyomo.Boolean, initialize = 0) # Discharging

In [95]:
# time indexed parameters
model.priceSell = pyomo.Param(model.Time, initialize = prices) # price for selling energy to grid
model.priceBuy = pyomo.Param(model.Time,  initialize = prices) # price for buying energy from grid

model.posLoad = pyomo.Param(model.Time, initialize = posLoadDict) # positive load
model.negLoad = pyomo.Param(model.Time, initialize = negLoadDict) # negative load

# single value parameters
model.etaCharge = pyomo.Param(initialize = batt.charging_efficiency) # charging efficiency
model.etaDischarge = pyomo.Param(initialize = batt.discharging_efficiency) # discharging efficiency
model.chargingLimit = pyomo.Param(initialize = batt.charging_power_limit) # charging limit (kW)
model.dischargingLimit = pyomo.Param(initialize = batt.discharging_power_limit) # discharging limit (kW)
model.bigM = pyomo.Param(initialize = 500000)  # Big M parameter

In [96]:
# Objective function - minimize cost of energy 
def obj_function(model):
    return sum((model.priceBuy[i] * model.posNetLoad[i]) + (model.priceSell[i] * model.negNetLoad[i]) for i in model.Time)


# Total Loss of Power Supply (TLPS)
# TLPS = Sum(S_failure[t for t in Time]) / T * 100 (%)
# or should i be S_failure[t] * net_load[t] for t in time 

In [97]:
# Constraints

# update state of charge 
def soc_rule(model, t):
    if t == 0:
        return (model.SOC[t] == model.posDeltaSOC[t] + model.negDeltaSOC[t])
    else:
        return (model.SOC[t] == model.SOC[t-1] + model.posDeltaSOC[t] + model.negDeltaSOC[t])
model.soc = pyomo.Constraint(model.Time, rule = soc_rule)

# ensures beginning and end storage match
def horizon_rule(model):
    return (model.SOC[T-1] == model.SOC[0])
model.horizon = pyomo.Constraint(rule = horizon_rule)

In [98]:
# Boolean constraints

# If battery is charging, then charging geq -large 
# If not, charging geq zero 
def bool_char_rule_1(model, i):
    return ((model.posDeltaSOC[i]) >= -model.bigM * (model.Bool_char[i]))
model.batt_charge_1 = pyomo.Constraint(model.Time, rule = bool_char_rule_1)

# If battery charging, then charging leq +large
# If not, charging leq zero 
def bool_char_rule_2(model, i):
    return ((model.posDeltaSOC[i]) <= 0 + model.bigM * (1 - model.Bool_dis[i]))

# If battery discharging, then discharge leq +large
# If not, discharge leq zero 
def bool_char_rule_3(model, i):
    return ((model.negDeltaSOC[i]) <= model.bigM * (model.Bool_dis[i]))

# If battery discharging, then discharge geq -large
# If not, discharge geq zero
def bool_char_rule_4(model, i):
    return ((model.negDeltaSOC[i]) >= 0 - model.bigM * (1 - model.Bool_char[i]))
            
# Battery can only charge or discharge at any given time
def batt_chat_dis(model, i):
    return (model.Bool_char[i] + model.Bool_dis[i] == 1)
model.batt_char_dis = pyomo.Constraint(model.Time, rule = batt_chat_dis)

In [ ]:
# # ensure charging rate obeyed
# def E_charging_rate_rule(model,t):
#     return (model.posEInGrid[t]+model.posEInPV[t])<=model.ChargingLimit
# model.chargingLimit_cons = pyomo.Constraint(model.Time, rule=E_charging_rate_rule)
# # ensure DIScharging rate obeyed
# def E_discharging_rate_rule(m,t):
#     return (model.negEOutLocal[t]+model.negEOutExport[t])>=model.DischargingLimit
# model.dischargingLimit_cons = pyomo.Constraint(model.Time, rule=E_discharging_rate_rule)

In [100]:
# ensures that the energy charged by the battery is equal to the energy coming into the battery from PV and the grid, respecting the charge efficiency
def pos_E_in_rule(model, i):
    return (model.posEInGrid[i] + model.posEInPV[i]) == model.posDeltaSOC[i]/model.etaCharge
model.posEIn_cons = pyomo.Constraint(model.Time, rule = pos_E_in_rule)

# ensures that energy discharged by the battery equals the energy going out to the grid and the local load, respecting the discharge efficiency
def neg_E_out_rule(model, i):
    return (model.negEOutLocal[i] + model.negEOutExport[i]) == model.negDeltaSOC[i]*model.etaDischarge
model.negEOut_cons = pyomo.Constraint(model.Time, rule = neg_E_out_rule)

# ensures that the energy coming into the battery from PV (posEInPV) does not exceed the available excess PV (negative net load)
def E_solar_charging_rule(model, i):
    return model.posEInPV[i] <= -model.negLoad[i]
model.E_solar_charging_cons = pyomo.Constraint(model.Time, rule = E_solar_charging_rule)

# ensures that the amount of energy discharged by the battery to the local load does not exceed the local load 
def E_local_discharge_rule(model, i):
    return model.negEOutLocal[i] >= -model.posLoad[i]
model.E_local_cons = pyomo.Constraint(model.Time, rule = E_local_discharge_rule)

# This calculates the net positive demand (i.e energy bought from grid): the net positive load is equal to the positive local load, PLUS the amount of energy charged by battery from grid, minus the amount of energy discharged by the battery to the local load
def E_pos_net_rule(model, i):
    return model.posNetLoad[i] == model.posLoad[i] + model.posEInGrid[i] + model.negEOutLocal[i]
model.E_posNet_cons = pyomo.Constraint(model.Time, rule = E_pos_net_rule)

# This calculates the net negative load (i.e. the energy sold to the grid): the net negative load is equal to the negative local load, PLUS the amount of energy taken from the PV by the battery, PLUS the amount of energy discharged by the battery to the grid 
def E_neg_net_rule(model, i):
    return model.negNetLoad[i] == model.negLoad[i] + model.posEInPV[i] + model.negEOutExport[i]
model.E_negNet_cons = pyomo.Constraint(model.Time, rule = E_neg_net_rule)

In [106]:
# run model
#opt = SolverFactory("gurobi", executable = "/usr/local/bin/gurobi_cl")
opt = SolverFactory("glpk", executable = "/usr/local/bin/glpsol")

In [107]:
results = opt.solve(model)

In [108]:
j = 0
for v in model.component_objects(pyomo.Var, active = True):
    print (j, v.getname())
    j+=1
    

0 SOC
1 posDeltaSOC
2 negDeltaSOC
3 posEInGrid
4 posEInPV
5 negEOutLocal
6 negEOutExport
7 posNetLoad
8 negNetLoad
9 Bool_char
10 Bool_dis


In [109]:
outputVars = np.zeros((9,len(prices)))

In [110]:
j = 0
for v in model.component_objects(pyomo.Var, active=True):
    print(v.getname())
    varobject = getattr(model, str(v))
    print (varobject.get_values())
    
    for index in varobject:
        outputVars[j,index] = varobject[index].value
    j+=1
    if j>=9:
        break

SOC
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.0, 23: 0.0}
posDeltaSOC
{0: 4.5, 1: 4.5, 2: 4.5, 3: 4.5, 4: -0.0, 5: 4.5, 6: -0.0, 7: 4.5, 8: 4.5, 9: 4.5, 10: -0.0, 11: 4.5, 12: 4.5, 13: 4.5, 14: -0.0, 15: 4.5, 16: 4.5, 17: 4.5, 18: 4.5, 19: -0.0, 20: 4.5, 21: -0.0, 22: 4.5, 23: 4.5}
negDeltaSOC
{0: -4.5, 1: -4.5, 2: -4.5, 3: -4.5, 4: -0.0, 5: -4.5, 6: -0.0, 7: -4.5, 8: -4.5, 9: -4.5, 10: -0.0, 11: -4.5, 12: -4.5, 13: -4.5, 14: -0.0, 15: -4.5, 16: -4.5, 17: -4.5, 18: -4.5, 19: -0.0, 20: -4.5, 21: -0.0, 22: -4.5, 23: -4.5}
posEInGrid
{0: 5.0, 1: 5.0, 2: 5.0, 3: 5.0, 4: 0.0, 5: 5.0, 6: 0.0, 7: 5.0, 8: 5.0, 9: 5.0, 10: 0.0, 11: 5.0, 12: 5.0, 13: 5.0, 14: 0.0, 15: 5.0, 16: 5.0, 17: 5.0, 18: 5.0, 19: 0.0, 20: 5.0, 21: 0.0, 22: 5.0, 23: 5.0}
posEInPV
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 

In [ ]:
#get the value of the objective function
model.obj_function

In [82]:
# get the total cost
cost_without_batt = np.sum([(buyPrice[i]*posLoad[i]/1000 + sellPrice[i]*negLoad[i]/1000) for i in range(len(buyPrice))])
cost_with_batt = np.sum([(buyPrice[i]*outputVars[7,i]/1000 + sellPrice[i]*outputVars[8,i]/1000) for i in range(len(buyPrice))])

print('Cost without battery:', cost_without_batt)
print('Cost with battery:', cost_with_batt)
print('Score: %.4f'%((cost_with_batt - cost_without_batt) / np.abs(cost_without_batt)))

NameError: name 'buyPrice' is not defined

# Model 2

In [ ]:
# Initialize model
model = pyomo.ConcreteModel()
model.Time = pyomo.RangeSet(0, T-1)

# Variables
model.SOC = pyomo.Var(model.Time, domain=pyomo.NonNegativeReals, bounds=(batt.min_SOC, batt.max_SOC))
model.Discharge = pyomo.Var(model.Time, domain=pyomo.NonNegativeReals, bounds=(0, batt.max_discharge))
model.Charge = pyomo.Var(model.Time, domain=pyomo.NonNegativeReals, bounds=(0, batt.max_charge))

model.EnergyFromGrid = pyomo.Var(model.Time, domain=pyomo.NonNegativeReals)
model.EnergyFromPV = pyomo.Var(model.Time, domain=pyomo.NonNegativeReals)

model.EnergyToGrid = pyomo.Var(model.Time, domain=pyomo.NonNegativeReals)
model.EnergyToLocalLoad = pyomo.Var(model.Time, domain=pyomo.NonNegativeReals)

# Bool Vars
model.CharegFromPV_Bool = pyomo.Var(model.Time, domain=pyomo.Binary)
model.ChargeFromGrid_Bool = pyomo.Var(model.Time, domain=pyomo.Binary)
model.Discharge_Bool = pyomo.Var(model.Time, domain=pyomo.Binary)


# Parameters
model.PV = pyomo.Param(model.Time, initialize=lambda m, t: pv[t])
model.PosLoad = pyomo.Param(model.Time, initialize=lambda m, t: load[t])
model.NegLoad =
model.BuyPrice = pyomo.Param(model.Time, initialize=lambda m, t: buyPrice[t])
model.SellPrice = pyomo.Param(model.Time, initialize=lambda m, t: sellPrice[t])

# Objective
def obj_rule(model):
    return sum(model.BuyPrice[t] * model.Load[t] - model.SellPrice[t] * model.PV[t] for t in model.Time)
model.obj_function = pyomo.Objective(rule=obj_rule, sense=pyomo.minimize)

# Constraints
def SOC_rule(model, t):
    if t == 0:
        return model.SOE[t] == batt.capacity * 0.50 # initialize battery at 50% capacity 
    else:
        return model.SOE[t] == model.SOE[t-1] + (batt.charging_efficiency * model.Charge[t]) - (model.Discharge[t] / batt.discharging_efficiency)
model.SOC_cons = pyomo.Constraint(model.Time, rule=SOC_rule)


def Charge_rule(model, t):
    return model.Charge[t] == model.Charge_bool[t] * (model.EnergyFromGrid[t] + model.EnergyFromPV[t])

def Discharge_limit_rule(model, t):
    return model.Discharge[t] <= batt.discharging_power_limit
model.Discharge_cons = pyomo.Constraint(model.Time, rule=Discharge_limit_rule)

def Charge_limit_rule(model, t):
    return model.Charge[t] <= batt.charging_power_limit
model.Charge_cons = pyomo.Constraint(model.Time, rule=Charge_limit_rule)

def Horizon_rule(model, t):
    return model.SOE[0] == model.SOE[T-1]
model.Horizon_cons = pyomo.Constraint(model.Time, rule=Horizon_rule)


# Can only either charge from PV, charge from grid, or discharge at any given time 
def Charge_bool_rule(model, t):
    return model.ChargeFromPV_Bool[t] + model.ChargeFromGrid_Bool[t] + model.Discharge_Bool[t] <= 1 
model.Charge_bool_cons = pyomo.Constraint(model.Time, rule=Charge_bool_rule)

# Can only charge from PV if net load is negative
def ChargeFromPV_rule(model, t):
    if model.netLoad[t] < 0:
        return model.ChargeFromPV_Bool[t] == 1
    else:
        return model.ChargeFromPV_Bool[t] == 0
model.ChargeFromPV_cons = pyomo.Constraint(model.Time, rule=ChargeFromPV_rule)



# run model
opt = SolverFactory("gurobi", executable = "/usr/local/bin/gurobi_cl")
results = opt.solve(model)

# get the value of the objective function
model.obj_function

# get the total cost
cost_without_batt = np.sum([(buyPrice[i]*load[i]/1000 - sellPrice[i]*pv[i]/1000) for i in range(len(buyPrice))])

cost_with_batt = np.sum([(buyPrice[i]*model.Load[i]/1000 - sellPrice[i]*model.PV[i]/1000) for i in range(len(buyPrice))])

print('Cost without battery:', cost_without_batt)
print('Cost with battery:', cost_with_batt)

## Model 3

In [92]:
# IMPORT DATA FILES USED BY THIS NOTEBOOK
import os,  requests

file_links = [("data/Prices_DAM_ALTA2G_7_B1.csv", "https://ndcbe.github.io/CBE60499/data/Prices_DAM_ALTA2G_7_B1.csv")]

# This cell has been added by nbpages. Run this cell to download data files required for this notebook.

for filepath, fileurl in file_links:
    stem, filename = os.path.split(filepath)
    if stem:
        if not os.path.exists(stem):
            os.mkdir(stem)
    if not os.path.isfile(filepath):
        with open(filepath, 'wb') as f:
            response = requests.get(fileurl)
            f.write(response.content)

In [96]:
# Load the data file
ca_data = pd.read_csv('./data/Prices_DAM_ALTA2G_7_B1.csv',names=['price'])

# Print the first 10 rows
ca_data.head()

,price
0,36.757
1,34.924
2,33.389
3,32.035
4,33.694


# Previous work

In [ ]:


# create model
m = pyomo.ConcreteModel()

# Problem DATA
T = 24

Zmin = 0.0
Zmax = 2.0

Qmin = -1.0
Qmax = 1.0

# Generate prices, solar output and load signals
np.random.seed(42)
# P = np.random.rand(T)*5.0
# S = np.random.rand(T)
# L = np.random.rand(T)*2.0


# prob_false = 0.7
# prob_true = 0.3
# outage_hours = np.random.choice([False, True], size= T, p=[prob_false, prob_true])
net_load = np.random.rand(T) * 50.0 - np.random.rand(T) * 20.0
prices = np.random.rand(T)* 20.0 
feed_in_tariff = prices/2

# Indexes
times = range(T)
times_plus_1 = range(T+1)

# Parameters
m.net_load = pyomo.Param(times, initialize=lambda m, t: net_load[t])
m.prices = pyomo.Param(times, initialize=lambda m, t: prices[t])
#m.feed_in_tariff = pyomo.Param(times, initialize=lambda m, t: feed_in_tariff[t])

# Decisions variables
m.Q = pyomo.Var(times, domain=pyomo.Reals, bounds = (Qmin, Qmax))
m.Z = pyomo.Var(times_plus_1, domain=pyomo.NonNegativeReals, bounds = (Zmin, Zmax))
#m.Sell = pyomo.Var(times, domain=pyomo.NonNegativeReals)

# Objective
cost = sum(m.prices[t]*(m.net_load[t] - m.Q[t]) for t in times)
m.cost = pyomo.Objective(expr = cost, sense=pyomo.minimize)

# constraints
m.cons = pyomo.ConstraintList()
m.cons.add(m.Z[0] == 0.5*(Zmin + Zmax))

for t in times:
    # m.cons.add(pyomo.inequality(Qmin, m.Q[t], Qmax))
    # m.cons.add(pyomo.inequality(Zmin, m.Z[t], Zmax))
    m.cons.add(m.Z[t+1] == m.Z[t] - m.Q[t])
   
    
def charge_constraint_rule(m, t):
    if m.net_load[t] < 0:
        return m.net_load[t] - m.Q[t] <= 0
    else:
        return m.net_load[t] - m.Q[t] >= 0
m.charge_constraint = pyomo.Constraint(times, rule=charge_constraint_rule)

    

# solve
solver = pyomo.SolverFactory('glpk', executable = '/usr/local/bin/glpsol')

solver.solve(m)

# display results
print("Total cost =", m.cost(), ".")

for v in m.component_objects(pyomo.Var, active=True):
    print ("Variable component object",v)
    print ("Type of component object: ", str(type(v))[1:-1]) # Stripping <> for nbconvert
    varobject = getattr(m, str(v))
    print ("Type of object accessed via getattr: ", str(type(varobject))[1:-1])

    for index in varobject:
        print ("   ", index, varobject[index].value)

In [ ]:
import pyomo.environ as pyomo
import numpy as np

# create model
m = pyomo.ConcreteModel()

# Problem DATA
T = 24

Zmin = 0.0
Zmax = 50.0

Qmin = -25.0
Qmax = 25.0

# Generate prices, solar output and load signals
np.random.seed(42)
# P = np.random.rand(T)*5.0
# S = np.random.rand(T)
# L = np.random.rand(T)*2.0


# prob_false = 0.7
# prob_true = 0.3
# outage_hours = np.random.choice([False, True], size= T, p=[prob_false, prob_true])
net_load = np.random.rand(T) * 50.0 - np.random.rand(T) * 20.0
prices = np.random.rand(T)* 5.0
feed_in_tariff = np.random.rand(T)*0

# Indexes
times = range(T)
times_plus_1 = range(T+1)

# Parameters
m.net_load = pyomo.Param(times, initialize=lambda m, t: net_load[t]) # kW
m.prices = pyomo.Param(times, initialize=lambda m, t: prices[t]) # $/kWh
m.feed_in_tariff = pyomo.Param(times, initialize=lambda m, t: feed_in_tariff[t]) # $/kWh

# Decisions variables
m.Q = pyomo.Var(times, domain=pyomo.Reals, bounds = (Qmin, Qmax)) # kW
m.Z = pyomo.Var(times_plus_1, domain=pyomo.NonNegativeReals, bounds = (Zmin, Zmax)) # kWh
m.Sell = pyomo.Var(times, domain=pyomo.NonNegativeReals) # kW

# Objective
cost = sum(m.prices[t]*(m.net_load[t] - m.Q[t]) - (m.feed_in_tariff[t] * m.Sell[t]) for t in times)
m.cost = pyomo.Objective(expr = cost, sense=pyomo.minimize)

# Constraints
def initial_state_of_energy(m, t):
    return m.Z[0] == 0
m.initial_state_of_energy = pyomo.Constraint(times, rule=initial_state_of_energy)

def update_state_of_energy(m, t):
    return m.Z[t+1] == m.Z[t] - m.Q[t] # m.Q is positive when discharging and negative when charging 
m.update_state_of_energy = pyomo.Constraint(times, rule=update_state_of_energy)

def charge_constraint_rule(m, t):
    if m.net_load[t] < 0: # if net load is negative, we charge the battery as much as we can, and sell the rest to the grid 
        return m.net_load[t] - m.Q[t] + m.Sell[t] <= 0
    else:
        return m.net_load[t] - m.Q[t] >= 0 # if net load is positive, we can only discharge the battery [positive net load minus positive discharge]
m.charge_constraint = pyomo.Constraint(times, rule=charge_constraint_rule)

def when_discharge(m, t):
    if m.net_load[t] > 0:
        return m.Q[t] >= 0
    else:
        return m.Q[t] <= 0
m.when_discharge = pyomo.Constraint(times, rule=when_discharge)

def only_sell_excess(m, t):
    if m.net_load[t] > 0:
        return m.Sell[t] == 0
    else:
        return m.Sell[t] >= 0
m.only_sell_excess = pyomo.Constraint(times, rule=only_sell_excess)


# Solve
solver = pyomo.SolverFactory('gurobi', executable = '/usr/local/bin/gurobi_cl')

solver.solve(m)

#display results
print("Total cost =", m.cost(), ".")

for i in range(len(m.Q)):
    print(round(m.net_load[i], 1), pyomo.value(m.Z[i]), pyomo.value(m.Q[i]), round(pyomo.value(m.Sell[i]),1))

# for v in m.component_objects(pyomo.Var, active=True):
#     print ("Variable component object",v)
#     print ("Type of component object: ", str(type(v))[1:-1]) # Stripping <> for nbconvert
#     varobject = getattr(m, str(v))
#     print ("Type of object accessed via getattr: ", str(type(varobject))[1:-1])

#     for index in varobject:
#         print ("   ", index, varobject[index].value)
        
        

In [ ]:
import pyomo.environ as pyomo
import numpy as np

# create model
m = pyomo.ConcreteModel()

# Problem DATA
T = 24

Zmin = 0.0
Zmax = 2.0

Qmin = -1.0
Qmax = 1.0

# Generate prices, solar output and load signals
np.random.seed(42)
# P = np.random.rand(T)*5.0
# S = np.random.rand(T)
# L = np.random.rand(T)*2.0


# prob_false = 0.7
# prob_true = 0.3
# outage_hours = np.random.choice([False, True], size= T, p=[prob_false, prob_true])
net_load = np.random.rand(T) * 50.0 - np.random.rand(T) * 20.0
prices = np.random.rand(T)* 20.0 
feed_in_tariff = np.random.rand(T)* 5.0 

# Indexes
times = range(T)
times_plus_1 = range(T+1)

# Parameters
m.net_load = pyomo.Param(times, initialize=lambda m, t: net_load[t]) # kW
m.prices = pyomo.Param(times, initialize=lambda m, t: prices[t]) # $/kWh
m.feed_in_tariff = pyomo.Param(times, initialize=lambda m, t: feed_in_tariff[t]) # $/kWh

# Decisions variables
m.Q = pyomo.Var(times, domain=pyomo.Reals, bounds = (Qmin, Qmax)) # kW
m.Z = pyomo.Var(times_plus_1, domain=pyomo.NonNegativeReals, bounds = (Zmin, Zmax)) # kWh
m.Sell = pyomo.Var(times, domain=pyomo.NonNegativeReals) # kW

# Objective
cost = sum(m.prices[t]*(m.net_load[t] - m.Q[t]) - (m.feed_in_tariff[t] * m.Sell[t]) for t in times)
m.cost = pyomo.Objective(expr = cost, sense=pyomo.minimize)

# Constraints
def initial_state_of_energy(m, t):
    return m.Z[0] == 0
m.initial_state_of_energy = pyomo.Constraint(times, rule=initial_state_of_energy)

def update_state_of_energy(m, t):
    return m.Z[t+1] == m.Z[t] - m.Q[t] # m.Q is positive when discharging and negative when charging 
m.update_state_of_energy = pyomo.Constraint(times, rule=update_state_of_energy)

def charge_constraint_rule(m, t):
    if m.net_load[t] < 0: # if net load is negative, we can only charge the battery, and sell the rest to the grid 
        return m.net_load[t] - m.Q[t] + m.Sell[t] == 0
    else:
        return m.net_load[t] - m.Q[t] >= 0 # if net load is positive, we can only discharge the battery [positive net load minus positive discharge]
m.charge_constraint = pyomo.Constraint(times, rule=charge_constraint_rule)

def when_discharge(m, t):
    if m.net_load[t] > 0:
        return m.Q[t] >= 0
    else:
        return m.Q[t] <= 0
m.when_discharge = pyomo.Constraint(times, rule=when_discharge)

def only_sell_excess(m, t):
    if m.net_load[t] > 0:
        return m.Sell[t] == 0
    else:
        return m.Sell[t] >= 0
m.only_sell_excess = pyomo.Constraint(times, rule=only_sell_excess)


# Solve
solver = pyomo.SolverFactory('glpk', executable = '/usr/local/bin/glpsol')

solver.solve(m)

#display results
print("Total cost =", m.cost(), ".")

for i in range(len(m.Q)):
    print(round(m.net_load[i], 1), pyomo.value(m.Z[i]), pyomo.value(m.Q[i]), round(pyomo.value(m.Sell[i]),1))

# for v in m.component_objects(pyomo.Var, active=True):
#     print ("Variable component object",v)
#     print ("Type of component object: ", str(type(v))[1:-1]) # Stripping <> for nbconvert
#     varobject = getattr(m, str(v))
#     print ("Type of object accessed via getattr: ", str(type(varobject))[1:-1])

#     for index in varobject:
#         print ("   ", index, varobject[index].value)
        
        

In [ ]:
import pyomo.environ as pyomo 

# Original Data
outage_hours = [True, True, False, False, True, False, True, True, False, False, False, False, False, True]
net_load_profile = np.random.uniform(-50, 50, len(outage_hours))
time = range(len(outage_hours))
prices = np.random.uniform(0, 10, len(outage_hours))

unserved_load = np.array([net_load_profile[i] if outage_hours[i] else 0 for i in range(len(outage_hours))])

# Create model
m = ConcreteModel()

# Sets
m.time = Set(initialize = [t for t in time])
m.outage_hours = Param(m.time, initialize = outage_hours)
m.net_load_profile = Param(m.time, initialize = net_load_profile)
#m.prices = Param(m.time, initialize = prices)
m.unserved_load = Param(m.time, initialize = unserved_load, mutable = True)

# Static parameters
m.soe_initial = 0 # kWh (state of energy)
m.capacity = 100 # kWh
m.duration = 4 # hours
m.rating = m.capacity / m.duration # kW
m.charge_efficiency = 0.9
m.discharge_efficiency = 0.9

# Variables
m.charge = Var(m.time, domain = NonNegativeReals, bounds = (0, m.rating)) # energy charged to the battery system from the PV (kW)
m.discharge = Var(m.time, domain = NonNegativeReals, bounds = (0, m.rating)) # energy discharged from the battery system to the load (kW)
m.SOE = Var(m.time, domain = NonNegativeReals, bounds = (0, m.capacity)) # state of energy of the battery system (kWh)
#m.unserved_load = Var(m.time, domain = NonNegativeReals) # amount of unserved load (kW)

# Objective Function - Minimize unserved load 
# def obj_expression(m):
#     return sum(m.unserved_load[t] for t in m.time)
# m.obj = Objective(rule = obj_expression, sense = minimize) 

def obj_expression(m):
    return sum(m.prices[t] * (m.net_load_profile[t] - m.discharge[t]) for t in m.time)
m.obj = Objective(rule = obj_expression, sense = minimize) 

# Constraints 
# def unserved_load_rule(m, t): # to calculate the amount of unserved load at each time step
#     if net_load_profile[t] > 0:
#         if m.outage_hours[t] == True:
#             return m.unserved_load[t] == m.net_load_profile[t] - m.discharge[t]
#         else: # if no outage at that time step, load is served by grid
#             return m.unserved_load[t] == 0
#     else: # if net load is negative, there is no load to serve anyway 
#         return m.unserved_load[t] == 0
# m.unserved_load_rule = Constraint(m.time, rule=unserved_load_rule)



def soe_start_rule(m): # to set the initial state of energy stored at the battery system
    return m.SOE[0] == m.soe_initial
m.soe_start_rule = Constraint(rule=soe_start_rule)

def state_of_energy(m,t):  # to connect the states of the amount of energy stored at, energy discharged from, energy charged to, the battery system
    if t== 0:
        return m.SOE[t] == 0
    else:
        return m.SOE[t] == m.SOE[t-1] + m.charge[t-1] - m.discharge[t-1] 
m.state_of_energy_rule = Constraint(m.time, rule=state_of_energy)

def energy_flow_direction(m,t): # to ensure that the battery only discharges when net load is positive and charges when net load is negative 
    if m.net_load_profile[t] >= 0:
        return m.charge[t] == 0
    else:
        return m.discharge[t] == 0

def available_to_discharge(m, t): # to ensure that the amount of energy discharged from the battery system to the load is less than the amount of energy stored at the battery system
    return m.discharge[t] <= m.SOE[t]
m.available_to_discharge = Constraint(m.time, rule=available_to_discharge)

def demand_for_discharge(m, t): # to ensure that the amount of energy discharged from the battery system to the load is less than the amount of energy needed by the load
    return m.discharge[t] <= m.net_load_profile[t]
m.demand_for_discharge = Constraint(m.time, rule=demand_for_discharge)


def room_to_charge(m, t): # to ensure that the amount of energy charged to the battery system from the PV is less than the amount of energy stored at the battery system
    return m.charge[t] <= m.capacity - m.SOE[t]
m.room_to_charge = Constraint(m.time, rule=room_to_charge)

def available_to_charge(m, t): # to ensure that the amount of energy charged to the battery system from the PV is less than the amount of energy available from the PV
    #note: this is negative because battery charges when net load is negative 
    return m.charge[t] <= -m.net_load_profile[t]
m.available_to_charge = Constraint(m.time, rule=available_to_charge)



# Solve
solver = pyomo.SolverFactory('glpk', executable = '/usr/local/bin/glpsol')
solver.solve(m)
